In [25]:
import pandas as pd
import numpy as np
import re
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import img_to_array, load_img

In [26]:
df = pd.read_csv('urbansounds_features.csv')

### Check interval of the values per column

In [27]:
column_intervals = df.describe().loc[['min', 'max']]
column_intervals

,chroma_stft,chroma_cqt,chroma_cens,melspectogram,rms,centroid,bandwidth,contrast,flatness,rolloff,...,mcffs_31,mcffs_32,mcffs_33,mcffs_34,mcffs_35,mcffs_36,mcffs_37,mcffs_38,mcffs_39,mcffs_40
min,0.008760,0.118011,0.126660,0.017318,0.001788,49.252793,66.012024,3.938343,1.648695e-07,81.951002,...,-20.268456,-12.144473,-19.343061,-28.116508,-16.987791,-14.576315,-36.001620,-23.080887,-25.212177,-13.395362
max,0.845297,0.796146,0.286841,65.208150,0.579545,10907.707763,7532.173550,29.609336,9.798845e-01,18628.717306,...,19.353420,19.055628,15.425904,22.999128,20.503590,26.710932,23.912758,17.683758,17.977505,30.761436


In [28]:
print(df.dtypes)

chroma_stft          float64
chroma_cqt           float64
chroma_cens          float64
melspectogram        float64
rms                  float64
centroid             float64
bandwidth            float64
contrast             float64
flatness             float64
rolloff              float64
crossing_rate        float64
tempogram            float64
fourier_tempogram     object
mcffs_1              float64
mcffs_2              float64
mcffs_3              float64
mcffs_4              float64
mcffs_5              float64
mcffs_6              float64
mcffs_7              float64
mcffs_8              float64
mcffs_9              float64
mcffs_10             float64
mcffs_11             float64
mcffs_12             float64
mcffs_13             float64
mcffs_14             float64
mcffs_15             float64
mcffs_16             float64
mcffs_17             float64
mcffs_18             float64
mcffs_19             float64
mcffs_20             float64
mcffs_21             float64
mcffs_22      

In [29]:
df.isnull().sum()

chroma_stft          0
chroma_cqt           0
chroma_cens          0
melspectogram        0
rms                  0
centroid             0
bandwidth            0
contrast             0
flatness             0
rolloff              0
crossing_rate        0
tempogram            0
fourier_tempogram    0
mcffs_1              0
mcffs_2              0
mcffs_3              0
mcffs_4              0
mcffs_5              0
mcffs_6              0
mcffs_7              0
mcffs_8              0
mcffs_9              0
mcffs_10             0
mcffs_11             0
mcffs_12             0
mcffs_13             0
mcffs_14             0
mcffs_15             0
mcffs_16             0
mcffs_17             0
mcffs_18             0
mcffs_19             0
mcffs_20             0
mcffs_21             0
mcffs_22             0
mcffs_23             0
mcffs_24             0
mcffs_25             0
mcffs_26             0
mcffs_27             0
mcffs_28             0
mcffs_29             0
mcffs_30             0
mcffs_31   

### CNN

In [30]:
def calculate_mean_from_string(string):
    cleaned_string = string.replace('\n', '')
    numbers = re.findall(r"[-+]?\d*\.\d+|\d+", cleaned_string)
    array = np.array(numbers, dtype=float)
    mean_value = np.mean(array)
    return mean_value


for column in df.columns:
    if column != 'Label':
        if df[column].dtype != float and df[column].dtype != int:
            df[column] = df[column].apply(calculate_mean_from_string)
    else:
        df[column] = df[column].str.split('-').str[1].astype(int)

In [31]:
# Classification
X = df.drop('Label', axis=1)
y = df['Label']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [32]:
# Print class distribution before SMOTE
print("Class distribution before SMOTE:")
print(pd.Series(y_train).value_counts())

Class distribution before SMOTE:
Label
9    811
5    807
2    803
4    798
7    791
0    789
3    783
8    749
1    352
6    302
Name: count, dtype: int64


In [33]:
# Oversample the features values using SMOTE
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

In [34]:
# Print class distribution after SMOTE
print("\nClass distribution after SMOTE:")
print(pd.Series(y_train_resampled).value_counts())


Class distribution after SMOTE:
Label
2    811
9    811
5    811
1    811
7    811
0    811
8    811
4    811
6    811
3    811
Name: count, dtype: int64


In [35]:
# Standardize the feature values
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_resampled)
X_test_scaled = scaler.transform(X_test)

In [36]:
# Encode labels to numerical values
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train_resampled)
y_test_encoded = label_encoder.transform(y_test)

In [37]:
# Reshape data for CNN
X_train_reshaped = X_train_scaled.reshape((X_train_scaled.shape[0], 1, X_train_scaled.shape[1], 1))
X_test_reshaped = X_test_scaled.reshape((X_test_scaled.shape[0], 1, X_test_scaled.shape[1], 1))

In [38]:
# Convert labels to categorical one-hot encoding
y_train_onehot = to_categorical(y_train_encoded)
y_test_onehot = to_categorical(y_test_encoded)

In [39]:
# Define the CNN model with different activation functions for hidden layers
activation_functions = ['relu', 'sigmoid']

for activation1 in activation_functions:
    for activation2 in activation_functions:
        # Define the CNN model
        model = Sequential()
        model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(1, X_train_scaled.shape[1], 1)))
        model.add(MaxPooling2D((2, 2)))
        model.add(Conv2D(64, (3, 3), activation=activation1))
        model.add(MaxPooling2D((2, 2)))
        model.add(Conv2D(64, (3, 3), activation=activation2))
        model.add(Flatten())
        model.add(Dense(64, activation=activation1))
        model.add(Dense(y_train_onehot.shape[1], activation='sigmoid'))  # Sigmoid for the output layer

ValueError: One of the dimensions in the output is <= 0 due to downsampling in conv2d_1. Consider increasing the input size. Received input shape [None, 1, 53, 1] which would produce output shape with a zero or negative value in a dimension.

In [ ]:
# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# Train the model
model.fit(X_train_reshaped, y_train_onehot, epochs=10, validation_split=0.2)

In [ ]:
# Evaluate the model on the test set
test_loss, test_acc = model.evaluate(X_test_reshaped, y_test_onehot)
print(f'Test accuracy: {test_acc}')